This notebook demonstrates that the XRootD issue when running on a distributed cluster does not show up when running locally.

## Contents

The first 4 cells set up the code, with some configurable parameters and code setup. The core of the code is in the processor cell defining the TriggerThreshold class, which is set up to just count the number of events in each file, and also add one to a counter for every chunk encountered.

After that, we create a local and distributed executor and runner, and then run the code to see the different outcomes.

In [1]:
from dask.distributed import Client
import os
from dask_jobqueue import HTCondorCluster
import awkward as ak
import coffea.processor as processor
from hist import Hist, axis
from coffea.analysis_tools import PackedSelection
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory
from coffea.nanoevents.methods import nanoaod

In [2]:
MAX_WORKERS = 40
MAX_CHUNKS = None
CHUNKSIZE = 5000

In [3]:
redirector = "root://cmsxrootd.fnal.gov//"
#redirector = "root://cmsxrootd.fnal.gov//store/test/xrootd/T1_US_FNAL/"
#redirector = "root://cms-xrd-global.cern.ch//store/test/xrootd/T2_CH_CERN/"

ll_fileset = {
    "ggZH_HToBB_ZToLL": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ggZH_HToBB_ZToLL_M-125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2520000/1412A75E-6CBA-5142-914D-2FE8356F680C.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ggZH_HToBB_ZToLL_M-125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2520000/7D3C5290-C72C-B140-9233-D7DA01D4B183.root"
    ],
    "ZH_HToBB_ZToLL": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZH_HToBB_ZToLL_M-125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2520000/56097D21-719D-564E-8E4A-C65214F252F9.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZH_HToBB_ZToLL_M-125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2820000/FE6AF237-7A43-C640-A0C0-672C18B80526.root"
    ]
}

background_fileset1 = {
    "Z1Jets_NuNu_ZpT_50To150_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-50To150_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/0F8CC648-9AF6-B044-802A-0EB7C835E0A2.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-50To150_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/28D97AEC-59D5-B740-A770-93FE1BAEE2B4.root"
    ],
    "Z1Jets_NuNu_ZpT_150To250_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-150To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/172EBACB-6262-9F4B-A0F4-2FAAFDE0C5E4.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-150To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/330D8B69-4B47-1E4B-A887-693EBC256F72.root"
    ],
    "Z1Jets_NuNu_ZpT_250To400_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/0125BE86-EAB7-0A41-991C-0AAAE0E1B411.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/214C20C9-FE96-624E-B066-9E93A3ECCCC6.root"
    ],
    "Z1Jets_NuNu_ZpT_400Toinf_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-400ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2430000/D96F633E-4E9F-E840-9D6A-7A0F2C9A4782.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z1JetsToNuNu_M-50_LHEFilterPtZ-400ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2500000/2D1166D3-D36E-FE48-831B-3E5129DF86B2.root"
    ],
    "Z2Jets_NuNu_ZpT_50To150_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-50To150_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/260000/139DD796-FB80-FB4C-832C-32910737F15C.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-50To150_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/260000/1533FD48-0478-D046-9619-984D2795AB13.root"
    ],
    "Z2Jets_NuNu_ZpT_150To250_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-150To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/4258322E-C506-D948-A741-8215643FA65B.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-150To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2520000/8192962A-B27D-A84D-84BE-56308098E439.root"
    ],
    "Z2Jets_NuNu_ZpT_250To400_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/110000/0578AAC5-73D9-2549-81AE-15780AA2AC09.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/110000/10F5455F-F6B7-574D-96AF-BB5BE84F0E96.root"
    ],
    "Z2Jets_NuNu_ZpT_400Toinf_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-400ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/250000/00F85493-F90B-614F-9A15-8FAC7F453087.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-400ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/250000/2EF80BFA-3194-D74C-A563-15A037A209DC.root"
    ]
}

background_fileset2 = {
    "DYJets_LL_HT_70To100_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-70to100_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/270000/088661DE-FACE-A94C-925A-54AAEF6F3DCF.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-70to100_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/270000/158E3EB5-BC86-5D4A-8FF5-9A56D48DF51F.root"
    ],
    "DYJets_LL_HT_100To200_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-100to200_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/270000/CBACD077-17BA-2F45-8D53-E85FA418AC38.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-100to200_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/0D6F4737-5496-CB46-96FF-0BF753D8BA6B.root"
    ],
    "DYJets_LL_HT_200To400_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-200to400_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/1DF66202-5953-FC43-947A-DAB20E014690.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-200to400_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/049A53D2-0293-F341-A519-D05EC501B569.root"
    ],
    "DYJets_LL_HT_600To800_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-600to800_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/A180B4E8-7AAA-A64B-B279-D21C2054AAFC.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-600to800_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/79AB1FF0-3ADB-C246-91EE-B9C14BC21CE8.root"
    ],
    "DYJets_LL_HT_1200To2500_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-1200to2500_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/26793660-5D04-C24B-813E-3C1744C84D2D.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-1200to2500_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/6275C5B3-2FF5-E64B-972A-795610435C10.root"
    ],
    "DYJets_LL_HT_400To600_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-400to600_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/91C0A1FE-594B-584F-830A-002AC104B529.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-400to600_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/00846A4F-433D-6D4E-8381-DD588DAE4954.root"
    ],
    "DYJets_LL_HT_800To1200_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-800to1200_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/021E8C2F-03FB-A043-AE3B-6868A0CB4D37.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-800to1200_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/0352722F-19DF-6D4D-8854-08F6E44F009D.root"
    ],
    "DYJets_LL_HT_2500Toinf_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-2500toInf_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/100000/0454549E-D951-B14A-B548-B59B5C0E95D7.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-2500toInf_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/110000/02076515-44A3-CE49-B9D6-B4CD7D05B0B9.root"
    ]
}

background_fileset4 = {
    "ST_tchannel_top_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_t-channel_top_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/0D659B3B-F2D9-8341-9EB1-1EFCD1807617.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_t-channel_top_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/11B6AA30-B615-0941-AAF0-164A70BE3350.root"
    ],
    "ST_tchannel_antitop_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_t-channel_antitop_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/8288F355-084C-0541-B8CD-EA2DE15FA214.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_t-channel_antitop_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/1307BC2B-75C3-2047-9F66-F6E6FEBCA11F.root"
    ],
    "ST_tW_top_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_tW_top_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/60FB6FDE-25EA-6C4E-80E5-96ED7EF8C294.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_tW_top_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/8D4046EF-A074-CC49-8B9C-4E75DDBAF3D0.root"
    ],
    "ST_tW_antitop_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/6371B801-DE75-5B4B-A781-98523A058E30.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/230000/872D46BD-1E52-D347-A96D-DCA176E252E5.root"
    ],
    "TTToSemiLeptonic_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0E9EA19A-AE0E-3149-88C3-D733240FF5AB.root"
    ],
    "TTToHadronic_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/0307C1DA-E49C-AB4B-9179-C70BE232321E.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/048D5B7E-A63F-554D-8386-4E8BFA761E62.root"
    ],
    "TTTo2L2Nu_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/0804DEBA-97D5-BE46-BB9D-B1125570966E.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/10671CD1-1127-0842-AE7F-AF20DA985C45.root"
    ],
    "WZ_2L2Q_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WZTo2Q2L_mllmin4p0_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/30000/AEFC36F1-A161-224D-B5F8-6FC8A64C6CD6.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WZTo2Q2L_mllmin4p0_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/80000/071B5530-E46B-324F-BC0A-EB039B421ECD.root"
    ],
    "WZ_3L1Nu_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2500000/2117CDEE-EE70-8F4A-8D0E-174A8F3F7CCA.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2500000/27AFDC22-E023-7444-9C16-735B40275131.root"
    ],
    "ZZ_2L2Q_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo2Q2L_mllmin4p0_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/A58AE71A-0273-8546-8FBD-3F729CAF6516.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo2Q2L_mllmin4p0_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/30000/164068D4-2101-A34C-BF4A-15AD715B04D4.root"
    ],
    "ZZ_4L_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo4L_TuneCP5_13TeV_powheg_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/6242D8A1-C702-B84C-BF5B-CB36A8B6C6D7.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo4L_TuneCP5_13TeV_powheg_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/110000/03F4D6CD-A6DF-AB4B-BC8A-442292B87ACF.root"
    ],
    "WW_2L2Nu_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WWTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/100000/DDE0E00A-D9CE-004B-B973-9F4BE582BE78.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WWTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/120000/5C849601-1B81-204C-A4C1-4D7F708507FF.root"
    ],
    "WZ_1L1Nu2Q_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WZTo1L1Nu2Q_4f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/60000/2089373A-9AF8-DC46-9142-1DC3DC97D3AE.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WZTo1L1Nu2Q_4f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/60000/5E47E4FC-3A70-064A-8938-1D9038FAD03D.root"
    ],
    "ZZ_2L2Nu_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo2L2Nu_TuneCP5_13TeV_powheg_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/0EA802F0-ACFA-2444-B77A-C3ED9061CE32.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo2L2Nu_TuneCP5_13TeV_powheg_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/0F83174C-034B-254C-B335-0582C26CA0EF.root"
    ],
    "ZZ_2Q2Nu_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo2Q2Nu_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/230000/22216217-0194-114A-8CD0-B069A89456D7.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/ZZTo2Q2Nu_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/230000/3EA019A0-9430-AB45-BC47-CE79EF0267F6.root"
    ],
    "WJets_LNu_WPt_100To250_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-100To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/100000/00448C1B-25DD-9840-9639-F9E0660C764A.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-100To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/100000/0061BFC8-C386-EB45-A5FE-45D7A9A699A9.root"
    ]
}

background_fileset5 = {
    "WJets_LNu_WPt_250To400_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/03236F17-AAB9-5A48-9679-D1DAF9CCAA90.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-250To400_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/0F8F0363-7055-3949-833A-68968B2CBEE7.root"
    ],
    "WJets_LNu_WPt_400To600_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-400To600_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/250000/002D75D6-1924-7B4B-AA94-361DB2FDFE50.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-400To600_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/250000/398C2A62-A9E5-594E-BD55-64D6360080A2.root"
    ],
    "WJets_LNu_WPt_600Toinf_18": [
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-600ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/14C0708C-9710-FB45-8A7B-E751D511B23E.root",
        redirector+"/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_Pt-600ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/1C6DA670-DC97-2F44-9342-385758AC01AF.root"
    ]
}

In [4]:
ELE_ISO_WP = 0.4 #from CMS AN -2019/229
MU_ISO_WP = 0.4 #from CMS AN -2019/229
BTAG_WP = 0.2 #baseline for now - slightly higher than highest "loose" WP in CMS AN -2019/229

class TriggerThreshold(processor.ProcessorABC):

    def __init__(self):
        #Initialize Coffea processor
        ak.behavior.update(nanoaod.behavior)

        ### Accumulator for holding histograms and value counts
        self._accumulator = {}

    @property
    def accumulator(self):
        return self._accumulator

    def process(self,events):
        output = {}
        output = processor.accumulate([output,self.process_shift(events)])

        output[events.metadata["dataset"]]["EventCount"] += 1

        return output

    def process_shift(self,events):

        dataset = events.metadata["dataset"]

        output = {}
        output[dataset] = {
            "EventCount": processor.value_accumulator(int),
            "Test": processor.value_accumulator(int)
        }
        
        output[dataset]["Test"] += 1

        return output

    def postprocess(self,accumulator):
        return accumulator

# Running

Now that we set up the processor and the filesets, we run the local and distributed executors, which for me shows the local executor succeeding, and the distributed executor failing with an XRootD error.

## Executor Creation
The next two cells create the local and distributed executors, respectively

In [5]:
Localexecutor = processor.IterativeExecutor(status=True)
print("Local executor created")

Local executor created


In [5]:
os.environ["CONDOR_CONFIG"] = "/etc/condor/condor_config"

try:
    _x509_localpath = (
        [
            line
            for line in os.popen("voms-proxy-info").read().split("\n")
            if line.startswith("path")
        ][0]
        .split(":")[-1]
        .strip()
    )
except Exception as err:
    raise RuntimeError(
        "x509 proxy could not be parsed, try creating it with 'voms-proxy-init'"
    ) from err
_x509_path = f'/scratch/{os.environ["USER"]}/{_x509_localpath.split("/")[-1]}'
os.system(f"cp {_x509_localpath} {_x509_path}")
_x509_path = os.path.basename(_x509_localpath)

#INPUT_FILES = os.path.join(os.getcwd(),'utils/helpers.py')
PWD = os.getcwd()
print(f'Transferring contents of {PWD} to workers')
INITIAL_DIR = f'/scratch/{os.environ["USER"]}'
cluster = HTCondorCluster(
    cores=1,
    memory="2 GB",
    disk="1 GB",
    death_timeout = '60',
    #python="/usr/local/bin/python3.8",
    job_extra_directives={
        "+JobFlavour": '"tomorrow"',
        "log": "dask_job_output.$(PROCESS).$(CLUSTER).log",
        "output": "dask_job_output.$(PROCESS).$(CLUSTER).out",
        "error": "dask_job_output.$(PROCESS).$(CLUSTER).err",
        "should_transfer_files": "yes",
        "when_to_transfer_output": "ON_EXIT_OR_EVICT",
        "+SingularityImage": '"/cvmfs/unpacked.cern.ch/registry.hub.docker.com/coffeateam/coffea-dask:0.7.22-py3.10-g7cbcc"',
        #"+SingularityImage": '"/cvmfs/unpacked.cern.ch/registry.hub.docker.com/coffeateam/coffea-dask:0.7.21-fastjet-3.4.0.1-g6238ea8"',
        #"+SingularityImage": '"/cvmfs/unpacked.cern.ch/registry.hub.docker.com/coffeateam/coffea-dask-cc7:0.7.21-fastjet-3.4.0.1-ge327427"',
        #"+SingularityImage": '"/cvmfs/unpacked.cern.ch/registry.hub.docker.com/coffeateam/coffea-dask:0.7.21-fastjet-3.4.0.1-gc3d707c"',
        "Requirements": "HasSingularityJobStart",
        #"request_GPUs" : "1",
        "InitialDir": INITIAL_DIR,
        "transfer_input_files": f'{_x509_path}'
        #"transfer_input_files": f'{_x509_path},{INPUT_FILES}'
        #"transfer_input_files": f'{_x509_path},{os.getcwd()}'
    },
    job_script_prologue=[
        "export XRD_RUNFORKHANDLER=1",
        f"export X509_USER_PROXY={_x509_path}",
    ]
)
print('Condor logs, output files, error files in {}'.format(INITIAL_DIR))
cluster.adapt(minimum=1, maximum=MAX_WORKERS)
client = Client(cluster)
Distexecutor = processor.DaskExecutor(client=client, status=True)
print("Distributed executor created")

client

Transferring contents of /afs/hep.wisc.edu/user/rsimeon/examples to workers
Condor logs, output files, error files in /scratch/rsimeon
Distributed executor created


Connection method: Cluster object,Cluster type: dask_jobqueue.HTCondorCluster
Dashboard: proxy/8787/status,
Dashboard: proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://144.92.181.248:27843,Workers: 0
Dashboard: proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Runners

These two cells create the local runner (using the local executor) and the distributed runner.

In [10]:
print("Creating local runner")
print("Using chunksize: {} and maxchunks {}".format(CHUNKSIZE,MAX_CHUNKS))
Localrunner = processor.Runner(
    executor=Localexecutor,
    schema=NanoAODSchema,
    chunksize=CHUNKSIZE,
    maxchunks=MAX_CHUNKS,
    skipbadfiles=True,
    xrootdtimeout=300,
)

Creating local runner
Using chunksize: 5000 and maxchunks None


In [6]:
print("Creating distributed runner")
print("Using chunksize: {} and maxchunks {}".format(CHUNKSIZE,MAX_CHUNKS))
Distrunner = processor.Runner(
    executor=Distexecutor,
    schema=NanoAODSchema,
    chunksize=CHUNKSIZE,
    maxchunks=MAX_CHUNKS,
    skipbadfiles=True,
    xrootdtimeout=300,
)

Creating distributed runner
Using chunksize: 5000 and maxchunks None


## Running the Runners

The next cell calls the local runner. It can take ~15 minutes to run, but it does succeed. The cell after that calls the distributed runner. It typically fails in under 10 minutes.

In [12]:
print("Calling runner")
local_results = Localrunner(
    background_fileset2,
    treename="Events",
    processor_instance=TriggerThreshold(),
)

Output()

Calling runner


Output()

In [7]:
print("Calling runner")
dist_results = Distrunner(
    background_fileset2,
    treename="Events",
    processor_instance=TriggerThreshold(),
)

Calling runner


AttributeError: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from '/opt/conda/lib/python3.8/site-packages/cloudpickle/cloudpickle.py'>

# Simple File Read

The local runner working shows that the error-raising file is actually readable. To back this up, the next cell does a bare-bones uproot read of the file. In fact, we can use Dask distributed workers to read the file successfully, and we can read the individual file with Coffea's NanoEventsFacotyr. It's only in the Coffea runner framework when trying to execute a processor that it seems to fail.

In [5]:
import uproot
file = "root://cmsxrootd.fnal.gov///store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-70to100_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/270000/158E3EB5-BC86-5D4A-8FF5-9A56D48DF51F.root"
with uproot.open({file: None}) as f:
    print(f.keys())
    print(type(f['Events;1']))

['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']
<class 'uproot.models.TTree.Model_TTree_v20'>


In [6]:
with uproot.open(file) as f:
    for future in client.map(lambda x: x * 5, range(4)):
        print(future.result())
        print(f.keys())

0
['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']
5
['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']
10
['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']
15
['tag;1', 'Events;1', 'LuminosityBlocks;1', 'Runs;1', 'MetaData;1', 'ParameterSets;1']


In [10]:
events = NanoEventsFactory.from_root(
    file,
    schemaclass=NanoAODSchema.v6,
).events()
print(len(events))

1720969


In [13]:
for future in client.map(lambda x: x * 5, range(4)):
    events = NanoEventsFactory.from_root(
        file,
        schemaclass=NanoAODSchema.v6,
    ).events()
    print(len(events))

1720969
1720969
1720969
1720969


In [10]:
client.shutdown()